# 🎯 Hull Tactical Market Prediction - API Inference Server

**Real-time prediction API for Kaggle competition**

## 📌 Setup Instructions

### Step 1: Upload Files to Kaggle Dataset

1. **Go to**: https://www.kaggle.com/datasets
2. **Click**: "New Dataset"
3. **Upload ALL files**:
   - All `.py` files from `src/` and `scripts/`
   - `params.yaml` from `conf/`
   - Trained model files from `artifacts/`
4. **Name it**: `mydata`
5. **Click**: "Create"

### Step 2: Configure Dataset Name

셀 3에서 데이터셋 이름 확인:

```python
DATASET_NAME = "mydata"  # 여기 확인!
```

### Step 3: Submit to Competition

1. **Competition Setting**: Enable "Internet" OFF
2. **Accelerator**: None (CPU)
3. **Click**: "Submit to Competition"
4. **Wait**: Real-time inference on hidden test set

## 🚀 How It Works:

1. **Load trained models** (return + risk models)
2. **Setup predict() function** for real-time inference
3. **Kaggle calls predict()** with each batch of test data
4. **Return allocation** in real-time (< 5 min per batch)

**Good luck! 🚀**

## 1️⃣ Setup: Configure Dataset Path

In [ ]:
import os
import sys
from pathlib import Path
import yaml

# ========== CONFIGURATION: 데이터셋 이름 (여기만 수정하세요!) ==========
DATASET_NAME = "mydata"
# ====================================================================

print("="*80)
print("SETTING UP PATHS")
print("="*80)

# Kaggle 데이터셋 경로
DATASET_PATH = Path(f"/kaggle/input/{DATASET_NAME}")

if DATASET_PATH.exists():
    sys.path.insert(0, str(DATASET_PATH))
    print(f"✓ Dataset found: {DATASET_PATH}")
    
    # ========== src 모듈 가상 생성 ==========
    import importlib.util
    import types
    
    src_module = types.ModuleType('src')
    src_module.__path__ = [str(DATASET_PATH)]
    sys.modules['src'] = src_module
    
    # 의존성 순서대로 모듈 로드
    py_files_to_load = [
        'utils', 'metric', 'cv', 'data', 'features', 'models', 
        'tuner', 'backtest', 'position', 'risk', 'interpretability', 
        'ensemble', 'timeseries_risk'
    ]
    
    for module_name in py_files_to_load:
        py_file = DATASET_PATH / f"{module_name}.py"
        if py_file.exists():
            full_module_name = f"src.{module_name}"
            spec = importlib.util.spec_from_file_location(full_module_name, py_file)
            if spec and spec.loader:
                module = importlib.util.module_from_spec(spec)
                sys.modules[full_module_name] = module
                setattr(src_module, module_name, module)
                try:
                    spec.loader.exec_module(module)
                    print(f"  ✓ Loaded: src.{module_name}")
                except Exception as e:
                    print(f"  ⚠️  Error loading src.{module_name}: {e}")
    
    print(f"\n✓ Created virtual 'src' module")
    
    # Working directory 설정
    os.chdir("/kaggle/working")
    print(f"✓ Working directory: {os.getcwd()}")
    
else:
    print(f"❌ Dataset not found: {DATASET_PATH}")
    input_dir = Path("/kaggle/input/")
    if input_dir.exists():
        for item in input_dir.iterdir():
            print(f"  📁 {item.name}")
    raise FileNotFoundError(f"Dataset '{DATASET_NAME}' not found!")

# ========== CONFIG 복사 ==========
config_path = DATASET_PATH / "params.yaml"
if config_path.exists():
    working_config_dir = Path("/kaggle/working/conf")
    working_config_dir.mkdir(exist_ok=True)
    
    import shutil
    shutil.copy(config_path, working_config_dir / "params.yaml")
    
    # conf 디렉토리를 Python path에 추가
    sys.path.insert(0, str(working_config_dir.parent))
    print(f"\n✓ Config copied to: {working_config_dir}/params.yaml")
else:
    print(f"\n⚠️  Config file not found: {config_path}")

print("\n✅ Setup complete!")

## 2️⃣ Load Trained Models

In [ ]:
print("="*80)
print("LOADING TRAINED MODELS")
print("="*80)

import pickle
import joblib
from pathlib import Path

# 모델 경로
MODEL_DIR = DATASET_PATH / "artifacts"

# Return model 로드
return_model_path = MODEL_DIR / "models_optimized" / "return_model.pkl"
if return_model_path.exists():
    with open(return_model_path, 'rb') as f:
        return_model = pickle.load(f)
    print(f"✓ Loaded return model from {return_model_path}")
else:
    print(f"⚠️  Return model not found: {return_model_path}")
    return_model = None

# Risk model 로드
risk_model_path = MODEL_DIR / "models_risk_optimized" / "risk_model.pkl"
if risk_model_path.exists():
    with open(risk_model_path, 'rb') as f:
        risk_model = pickle.load(f)
    print(f"✓ Loaded risk model from {risk_model_path}")
else:
    print(f"⚠️  Risk model not found: {risk_model_path}")
    risk_model = None

# Position strategy 파라미터 로드
strategy_path = MODEL_DIR / "position_strategy.json"
if strategy_path.exists():
    import json
    with open(strategy_path, 'r') as f:
        strategy_params = json.load(f)
    print(f"✓ Loaded position strategy from {strategy_path}")
else:
    print(f"⚠️  Position strategy not found: {strategy_path}")
    strategy_params = {}

print("\n✅ Model loading complete!")
print(f"  Return model: {'✓' if return_model else '✗'}")
print(f"  Risk model: {'✓' if risk_model else '✗'}")
print(f"  Strategy params: {'✓' if strategy_params else '✗'}")

## 3️⃣ Define Prediction Function

In [ ]:
import polars as pl
from src.features import FeatureEngineering
from src.position import SharpeScalingMapper

def predict(test: pl.DataFrame) -> float:
    """
    Real-time prediction function called by Kaggle API.
    
    Args:
        test: Polars DataFrame with batch of test features
        
    Returns:
        allocation: float between 0.0 and 2.0
    """
    try:
        # Convert to pandas for feature engineering
        test_pd = test.to_pandas()
        
        # Feature engineering
        fe = FeatureEngineering()
        test_features = fe.transform(test_pd)
        
        # Get selected features
        feature_cols = [col for col in test_features.columns 
                       if col not in ['date_id', 'return_1d', 'symbol']]
        
        # Predict return (r_hat)
        if return_model is not None:
            r_hat = return_model.predict(test_features[feature_cols])
        else:
            r_hat = 0.0
        
        # Predict risk (sigma_hat)
        if risk_model is not None:
            sigma_hat = risk_model.predict(test_features[feature_cols])
        else:
            sigma_hat = 0.1  # default risk
        
        # Map to position using strategy
        mapper = SharpeScalingMapper(**strategy_params)
        allocation = mapper.map(r_hat, sigma_hat)
        
        # Ensure within bounds [0, 2]
        allocation = max(0.0, min(2.0, float(allocation)))
        
        return allocation
        
    except Exception as e:
        print(f"⚠️  Prediction error: {e}")
        return 0.0  # Safe default

print("✅ Prediction function defined!")

## 4️⃣ Start Inference Server

In [ ]:
import os
import kaggle_evaluation.default_inference_server

print("="*80)
print("STARTING INFERENCE SERVER")
print("="*80)

# Create inference server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # Production: Serve API for hidden test set
    print("🚀 Running in COMPETITION mode - serving real-time predictions")
    inference_server.serve()
else:
    # Local testing: Run on public test set
    print("🧪 Running in LOCAL mode - testing on public data")
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

print("\n✅ Inference server started!")

## 📊 Submission Checklist

### 🎯 Before Submitting:

1. ✅ Train models locally and save to `artifacts/`
   - `artifacts/models_optimized/return_model.pkl`
   - `artifacts/models_risk_optimized/risk_model.pkl`
   - `artifacts/position_strategy.json`

2. ✅ Upload to Kaggle Dataset (`mydata`)
   - All `.py` files from `src/` and `scripts/`
   - `params.yaml` from `conf/`
   - `artifacts/` folder with trained models

3. ✅ Set `DATASET_NAME = "mydata"` in Cell 3

4. ✅ Competition Settings:
   - Internet: **OFF**
   - Accelerator: **None** (CPU)

5. ✅ Click "Submit to Competition"

### ⚡ How API Works:

```
Kaggle loads notebook → Load models → Start server → 
For each batch: Call predict() → Return allocation → Next batch
```

**Response time: < 5 minutes per batch**

**Good luck! 🚀**